# Data Analysis

## Data Preparation + Cleaning

### Loading and Viewing Data

In [621]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import LabelEncoder
import io
import json

In [622]:
# load in the dataset
dataset = pd.read_csv('../data/survey_responses.csv')
raw_dataset = dataset.copy()
# view the head of the dataset
dataset.head()

,Year_of_study,Field_of_study,Ever_used_platform,Time_investing,Main_platform_type,Confidence_in_terms,Purchasing_power_q,Single_stock_risk_q,Inflation_time_value_q,Knowledge_improved,Confidence_interpreting_data,Features_used_most,Regretted_misinterpretation,Frequency_based_on_app_data,Greater_influence,Apps_oversimplify,Likely_to_use_new_features,Data_viz_helps
0,2nd year,Humanities / Arts,Yes,1-2 years,Trading app,Very confident,It decreases,More risky than investing in a diversified fund,£110 in one year,Slightly,Not confident,Stock performance charts,Yes,Rarely,Data and analytics,Disagree,Very likely,Agree
1,2nd year,Engineering/ Science,No,6-12 months,Trading app,Not confident,Not sure,Not sure,£100 today,Significantly,Very confident,Other,No,Rarely,Data and analytics,Agree,Likely,Agree
2,3rd year,Engineering/ Science,No,1-2 years,Trading app,Somewhat confident,It decreases,More risky than investing in a diversified fund,£110 in one year,Significantly,Very confident,Portfolio analytics,No,Sometimes,Social media trends,Agree,Likely,Agree
3,1st year,Engineering/ Science,No,Less than 6 months,Other,Confident,It decreases,Not sure,£110 in one year,Slightly,Very confident,Portfolio analytics,No,Sometimes,Data and analytics,Disagree,Very likely,Disagree
4,3rd year,Business / Economics,No,N/A - Not an investor,N/A - Not an investor,Somewhat confident,It increases,More risky than investing in a diversified fund,£110 in one year,N/A - Not an investor,Very confident,Stock performance charts,No,Rarely,Social media trends,Agree,Very likely,Agree


In [623]:
# dataset info
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Year_of_study                 1000 non-null   object
 1   Field_of_study                1000 non-null   object
 2   Ever_used_platform            1000 non-null   object
 3   Time_investing                1000 non-null   object
 4   Main_platform_type            1000 non-null   object
 5   Confidence_in_terms           1000 non-null   object
 6   Purchasing_power_q            1000 non-null   object
 7   Single_stock_risk_q           1000 non-null   object
 8   Inflation_time_value_q        1000 non-null   object
 9   Knowledge_improved            1000 non-null   object
 10  Confidence_interpreting_data  1000 non-null   object
 11  Features_used_most            1000 non-null   object
 12  Regretted_misinterpretation   1000 non-null   object
 13  Frequency_based_on_

In [624]:
# dataset description + summary stats
dataset.describe()

,Year_of_study,Field_of_study,Ever_used_platform,Time_investing,Main_platform_type,Confidence_in_terms,Purchasing_power_q,Single_stock_risk_q,Inflation_time_value_q,Knowledge_improved,Confidence_interpreting_data,Features_used_most,Regretted_misinterpretation,Frequency_based_on_app_data,Greater_influence,Apps_oversimplify,Likely_to_use_new_features,Data_viz_helps
count,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
unique,4,4,2,5,4,4,4,4,4,5,4,5,3,5,4,5,5,5
top,2nd year,Business / Economics,Yes,N/A - Not an investor,Trading app,Confident,It decreases,More risky than investing in a diversified fund,£110 in one year,N/A - Not an investor,Confident,Stock performance charts,No,Sometimes,Data and analytics,Agree,Likely,Agree
freq,313,381,658,369,375,408,585,649,533,369,442,350,693,385,381,360,420,457


In [625]:
# look at unique responses to each column and the count of them
for column_name in dataset.columns:
    print(dataset[column_name].unique())
    print(dataset[column_name].value_counts())
    print("")

['2nd year' '3rd year' '1st year' 'Postgraduate']
Year_of_study
2nd year        313
3rd year        297
Postgraduate    206
1st year        184
Name: count, dtype: int64

['Humanities / Arts' 'Engineering/ Science' 'Business / Economics' 'Other']
Field_of_study
Business / Economics    381
Engineering/ Science    248
Humanities / Arts       225
Other                   146
Name: count, dtype: int64

['Yes' 'No']
Ever_used_platform
Yes    658
No     342
Name: count, dtype: int64

['1-2 years' '6-12 months' 'Less than 6 months' 'N/A - Not an investor'
 'Over 2 years']
Time_investing
N/A - Not an investor    369
Less than 6 months       187
6-12 months              179
1-2 years                161
Over 2 years             104
Name: count, dtype: int64

['Trading app' 'Other' 'N/A - Not an investor' 'Robo-advisor']
Main_platform_type
Trading app              375
N/A - Not an investor    369
Robo-advisor             195
Other                     61
Name: count, dtype: int64

['Very confident'

### Clean Section 1 - Background

In [626]:
# Clean Q1
study_map = {
    '1st year': 1,
    '2nd year': 2,
    '3rd year': 3,
    'Postgraduate': 4 
}

dataset['Year_of_study'] = dataset['Year_of_study'].map(study_map)

In [627]:
# Clean Q2
field_map = {
    'Business / Economics': 0,
    'Humanities / Arts': 1,
    'Engineering/ Science': 2,
    'Other': 3
}

dataset['Field_of_study'] = dataset['Field_of_study'].map(field_map)

In [628]:
# Clean Q3
use_platform_map = {
    'No': 0,
    'Yes': 1
}
dataset['Ever_used_platform'] = dataset['Ever_used_platform'].map(use_platform_map)

In [629]:
# Clean Q4
time_map = {
    'N/A - Not an investor': 0,
    'Less than 6 months': 1,
    '6-12 months': 2,
    '1-2 years': 3,
    'Over 2 years': 4
}

dataset['Time_investing'] = dataset['Time_investing'].map(time_map)

In [630]:
# Clean Q5
platform_map = {
    'N/A - Not an investor': 0,
    'Trading app': 1,
    'Robo-advisor': 2,
    'Other': 3
}

dataset['Main_platform_type'] = dataset['Main_platform_type'].map(platform_map)

In [631]:
dataset.head()

,Year_of_study,Field_of_study,Ever_used_platform,Time_investing,Main_platform_type,Confidence_in_terms,Purchasing_power_q,Single_stock_risk_q,Inflation_time_value_q,Knowledge_improved,Confidence_interpreting_data,Features_used_most,Regretted_misinterpretation,Frequency_based_on_app_data,Greater_influence,Apps_oversimplify,Likely_to_use_new_features,Data_viz_helps
0,2,1,1,3,1,Very confident,It decreases,More risky than investing in a diversified fund,£110 in one year,Slightly,Not confident,Stock performance charts,Yes,Rarely,Data and analytics,Disagree,Very likely,Agree
1,2,2,0,2,1,Not confident,Not sure,Not sure,£100 today,Significantly,Very confident,Other,No,Rarely,Data and analytics,Agree,Likely,Agree
2,3,2,0,3,1,Somewhat confident,It decreases,More risky than investing in a diversified fund,£110 in one year,Significantly,Very confident,Portfolio analytics,No,Sometimes,Social media trends,Agree,Likely,Agree
3,1,2,0,1,3,Confident,It decreases,Not sure,£110 in one year,Slightly,Very confident,Portfolio analytics,No,Sometimes,Data and analytics,Disagree,Very likely,Disagree
4,3,0,0,0,0,Somewhat confident,It increases,More risky than investing in a diversified fund,£110 in one year,N/A - Not an investor,Very confident,Stock performance charts,No,Rarely,Social media trends,Agree,Very likely,Agree


### Clean Section 2 - Financial Literacy

In [632]:
# define if they got they answer correct
columns_with_correct_answers = ['Purchasing_power_q', 
                                'Single_stock_risk_q', 
                                'Inflation_time_value_q']

correct_answers = ['It decreases',
                    'More risky than investing in a diversified fund',
                    '£110 in one year']

for column, correct_answer in zip(columns_with_correct_answers, correct_answers):
        
        dataset[column] = (dataset[column] == correct_answer).astype(int)
        
        print(f"Replaced values in column: '{column}'")

for column_name in columns_with_correct_answers:
    print(dataset[column_name].unique())
    print(dataset[column_name].value_counts())
    print("")

Replaced values in column: 'Purchasing_power_q'
Replaced values in column: 'Single_stock_risk_q'
Replaced values in column: 'Inflation_time_value_q'
[1 0]
Purchasing_power_q
1    585
0    415
Name: count, dtype: int64

[1 0]
Single_stock_risk_q
1    649
0    351
Name: count, dtype: int64

[1 0]
Inflation_time_value_q
1    533
0    467
Name: count, dtype: int64



In [633]:
columns_to_label_enc = ['Confidence_in_terms',
                        'Knowledge_improved']

confidence_map = {
        'Not confident': 0,
        'Somewhat confident': 1,
        'Confident': 2,
        'Very confident': 3
    }

knowledge_map_ = {
    'N/A - Not an investor': 0,  
    'No': 1,
    'Slightly': 2,
    'Moderately': 3,
    'Significantly': 4
}

dataset['Confidence_in_terms'] = dataset['Confidence_in_terms'].map(confidence_map)
dataset['Knowledge_improved'] = dataset['Knowledge_improved'].map(knowledge_map_)

for column_name in columns_to_label_enc:
    print(dataset[column_name].unique())
    print(dataset[column_name].value_counts())
    print("")

[3 0 1 2]
Confidence_in_terms
2    408
1    293
3    196
0    103
Name: count, dtype: int64

[2 4 0 1 3]
Knowledge_improved
0    369
3    207
2    190
4    137
1     97
Name: count, dtype: int64



In [634]:
# Calculate a financial literacy score from the survery results
columns_to_sum = ['Purchasing_power_q', 
                  'Single_stock_risk_q', 
                  'Inflation_time_value_q']

# 2. Sum them (axis=1) and assign the result to a new column
dataset['Financial_Literacy_Score'] = dataset[columns_to_sum].sum(axis=1)

In [635]:
dataset.head()

,Year_of_study,Field_of_study,Ever_used_platform,Time_investing,Main_platform_type,Confidence_in_terms,Purchasing_power_q,Single_stock_risk_q,Inflation_time_value_q,Knowledge_improved,Confidence_interpreting_data,Features_used_most,Regretted_misinterpretation,Frequency_based_on_app_data,Greater_influence,Apps_oversimplify,Likely_to_use_new_features,Data_viz_helps,Financial_Literacy_Score
0,2,1,1,3,1,3,1,1,1,2,Not confident,Stock performance charts,Yes,Rarely,Data and analytics,Disagree,Very likely,Agree,3
1,2,2,0,2,1,0,0,0,0,4,Very confident,Other,No,Rarely,Data and analytics,Agree,Likely,Agree,0
2,3,2,0,3,1,1,1,1,1,4,Very confident,Portfolio analytics,No,Sometimes,Social media trends,Agree,Likely,Agree,3
3,1,2,0,1,3,2,1,0,1,2,Very confident,Portfolio analytics,No,Sometimes,Data and analytics,Disagree,Very likely,Disagree,2
4,3,0,0,0,0,1,0,1,1,0,Very confident,Stock performance charts,No,Rarely,Social media trends,Agree,Very likely,Agree,2


### Clean Section 3 - Digital Literacy

In [636]:
dataset.head()

,Year_of_study,Field_of_study,Ever_used_platform,Time_investing,Main_platform_type,Confidence_in_terms,Purchasing_power_q,Single_stock_risk_q,Inflation_time_value_q,Knowledge_improved,Confidence_interpreting_data,Features_used_most,Regretted_misinterpretation,Frequency_based_on_app_data,Greater_influence,Apps_oversimplify,Likely_to_use_new_features,Data_viz_helps,Financial_Literacy_Score
0,2,1,1,3,1,3,1,1,1,2,Not confident,Stock performance charts,Yes,Rarely,Data and analytics,Disagree,Very likely,Agree,3
1,2,2,0,2,1,0,0,0,0,4,Very confident,Other,No,Rarely,Data and analytics,Agree,Likely,Agree,0
2,3,2,0,3,1,1,1,1,1,4,Very confident,Portfolio analytics,No,Sometimes,Social media trends,Agree,Likely,Agree,3
3,1,2,0,1,3,2,1,0,1,2,Very confident,Portfolio analytics,No,Sometimes,Data and analytics,Disagree,Very likely,Disagree,2
4,3,0,0,0,0,1,0,1,1,0,Very confident,Stock performance charts,No,Rarely,Social media trends,Agree,Very likely,Agree,2


In [637]:
# Clean Q11
interpreting_data_map = {
    'Not confident': 0,
    'Somewhat confident': 1,
    'Confident': 2, 
    'Very confident': 3
}

dataset['Confidence_interpreting_data'] = dataset['Confidence_interpreting_data'].map(interpreting_data_map)

In [638]:
# Clean Q12
use_map = {
    'Portfolio analytics': 0,
    'Stock performance charts': 1,
    'Market news': 2,
    'AI or data insights': 3,
    'Other': 4
}

dataset['Features_used_most'] = dataset['Features_used_most'].map(use_map)

In [639]:
# Clean Q13

regret_map = {
    'Not sure': 0,
    'No': 1, 
    'Yes': 2
}

dataset['Regretted_misinterpretation'] = dataset['Regretted_misinterpretation'].map(regret_map)

### Clean Section 4 - Investment Behaviour & Attitudes

In [640]:
dataset.head()

,Year_of_study,Field_of_study,Ever_used_platform,Time_investing,Main_platform_type,Confidence_in_terms,Purchasing_power_q,Single_stock_risk_q,Inflation_time_value_q,Knowledge_improved,Confidence_interpreting_data,Features_used_most,Regretted_misinterpretation,Frequency_based_on_app_data,Greater_influence,Apps_oversimplify,Likely_to_use_new_features,Data_viz_helps,Financial_Literacy_Score
0,2,1,1,3,1,3,1,1,1,2,0,1,2,Rarely,Data and analytics,Disagree,Very likely,Agree,3
1,2,2,0,2,1,0,0,0,0,4,3,4,1,Rarely,Data and analytics,Agree,Likely,Agree,0
2,3,2,0,3,1,1,1,1,1,4,3,0,1,Sometimes,Social media trends,Agree,Likely,Agree,3
3,1,2,0,1,3,2,1,0,1,2,3,0,1,Sometimes,Data and analytics,Disagree,Very likely,Disagree,2
4,3,0,0,0,0,1,0,1,1,0,3,1,1,Rarely,Social media trends,Agree,Very likely,Agree,2


In [641]:
# Clean Q14
frequency_map = {
    'Never': 0,
    'Rarely': 1,
    'Sometimes': 2,
    'Often': 3,
    'Always': 4
}
dataset['Frequency_based_on_app_data'] = dataset['Frequency_based_on_app_data'].map(frequency_map)

In [642]:
# Clean Q15 
influence_map = {
    'Data and analytics': 0,
    'Social media trends': 1, 
    'Advice from others': 2,
    'Personal decisions': 3
}

dataset['Greater_influence'] = dataset['Greater_influence'].map(influence_map)

In [643]:
# Clean Q16
oversimplify_map = {
    'Strongly disagree': 0,
    'Disagree': 1,
    'Neutral': 2,
    'Agree': 3,
    'Strongly agree': 4
}
dataset['Apps_oversimplify'] = dataset['Apps_oversimplify'].map(oversimplify_map)


In [644]:
# Clean Q17
likely_map = {
    'Very unlikely': 0,
    'Unlikely': 1,
    'Neutral': 2,
    'Likely': 3,
    'Very likely': 4
}
dataset['Likely_to_use_new_features'] = dataset['Likely_to_use_new_features'].map(likely_map)


In [645]:
# Clean Q18
viz_helps_map = {
    'Strongly disagree': 0,
    'Disagree': 1,
    'Neutral': 2,
    'Agree': 3,
    'Strongly agree': 4
}
dataset['Data_viz_helps'] = dataset['Data_viz_helps'].map(viz_helps_map)


In [646]:
# Create the digital literacy score
columns_for_score = [
        'Confidence_interpreting_data',
        'Data_viz_helps'                
    ]

dataset['Digital_Literacy_Score'] = dataset[columns_for_score].sum(axis=1)

## Analysis

In [647]:
raw_dataset.head()

,Year_of_study,Field_of_study,Ever_used_platform,Time_investing,Main_platform_type,Confidence_in_terms,Purchasing_power_q,Single_stock_risk_q,Inflation_time_value_q,Knowledge_improved,Confidence_interpreting_data,Features_used_most,Regretted_misinterpretation,Frequency_based_on_app_data,Greater_influence,Apps_oversimplify,Likely_to_use_new_features,Data_viz_helps
0,2nd year,Humanities / Arts,Yes,1-2 years,Trading app,Very confident,It decreases,More risky than investing in a diversified fund,£110 in one year,Slightly,Not confident,Stock performance charts,Yes,Rarely,Data and analytics,Disagree,Very likely,Agree
1,2nd year,Engineering/ Science,No,6-12 months,Trading app,Not confident,Not sure,Not sure,£100 today,Significantly,Very confident,Other,No,Rarely,Data and analytics,Agree,Likely,Agree
2,3rd year,Engineering/ Science,No,1-2 years,Trading app,Somewhat confident,It decreases,More risky than investing in a diversified fund,£110 in one year,Significantly,Very confident,Portfolio analytics,No,Sometimes,Social media trends,Agree,Likely,Agree
3,1st year,Engineering/ Science,No,Less than 6 months,Other,Confident,It decreases,Not sure,£110 in one year,Slightly,Very confident,Portfolio analytics,No,Sometimes,Data and analytics,Disagree,Very likely,Disagree
4,3rd year,Business / Economics,No,N/A - Not an investor,N/A - Not an investor,Somewhat confident,It increases,More risky than investing in a diversified fund,£110 in one year,N/A - Not an investor,Very confident,Stock performance charts,No,Rarely,Social media trends,Agree,Very likely,Agree


In [648]:
dataset.head()

,Year_of_study,Field_of_study,Ever_used_platform,Time_investing,Main_platform_type,Confidence_in_terms,Purchasing_power_q,Single_stock_risk_q,Inflation_time_value_q,Knowledge_improved,Confidence_interpreting_data,Features_used_most,Regretted_misinterpretation,Frequency_based_on_app_data,Greater_influence,Apps_oversimplify,Likely_to_use_new_features,Data_viz_helps,Financial_Literacy_Score,Digital_Literacy_Score
0,2,1,1,3,1,3,1,1,1,2,0,1,2,1,0,1,4,3,3,3
1,2,2,0,2,1,0,0,0,0,4,3,4,1,1,0,3,3,3,0,6
2,3,2,0,3,1,1,1,1,1,4,3,0,1,2,1,3,3,3,3,6
3,1,2,0,1,3,2,1,0,1,2,3,0,1,2,0,1,4,1,2,4
4,3,0,0,0,0,1,0,1,1,0,3,1,1,1,1,3,4,3,2,6
